<a href="https://colab.research.google.com/github/sumanjali19/devops-master-class/blob/master/AI_A__BFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# A* CODE with performance analysis
import heapq
import time
import tracemalloc

sld_bucharest = {
    'Arad': 366, 'Zerind': 374, 'Oradea': 380, 'Sibiu': 253, 'Timisoara': 329,
    'Lugoj': 244, 'Mehadia': 241, 'Drobeta': 242, 'Craiova': 160, 'Rimnicu Vilcea': 193,
    'Fagaras': 178, 'Pitesti': 98, 'Bucharest': 0, 'Giurgiu': 77, 'Urziceni': 80,
    'Hirsova': 151, 'Eforie': 161, 'Vaslui': 199, 'Iasi': 226, 'Neamt': 234
}


def a_star_search_with_multiple_heuristics(graph, start, goal, sld_bucharest, heuristic_type='sld', intermediary='Bucharest'):
    # Track node expansion
    node_expansion_count = 0

    # Initialize frontier (priority queue) and other variables
    def heuristic(city, goal):
        if heuristic_type == 'sld':  # Straight-line distance heuristic
            return sld_bucharest.get(city, float('inf'))
        elif heuristic_type == 'triangle':  # Triangle inequality heuristic
            return abs(sld_bucharest.get(city, float('inf')) - sld_bucharest.get(goal, float('inf')))

    frontier = [(0 + heuristic(start, goal), start)]
    heapq.heapify(frontier)  # Priority queue (min-heap)
    explored = set()
    g_costs = {start: 0}
    path = {start: None}

    while frontier:
        _, node = heapq.heappop(frontier)
        node_expansion_count += 1  # Increment node expansion count when a node is dequeued

        if node == goal:
            return construct_path(path, start, goal), g_costs[goal], node_expansion_count  # Return the path, cost, and node expansion

        explored.add(node)

        for neighbor, cost in graph[node].items():
            tentative_g = g_costs[node] + cost
            if neighbor not in explored or tentative_g < g_costs.get(neighbor, float('inf')):
                g_costs[neighbor] = tentative_g
                f_cost = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(frontier, (f_cost, neighbor))
                path[neighbor] = node

    return None, float('inf'), node_expansion_count  # Return no path and infinite cost if goal not found


def construct_path(path, start, goal):
    """Helper function to reconstruct the path from start to goal."""
    route = []
    node = goal
    while node is not None:
        route.append(node)
        node = path[node]
    route.reverse()
    return route


def analyze_a_star_performance(graph, start, goal, sld_bucharest, heuristic_type='sld'):
    # Track memory usage using tracemalloc
    tracemalloc.start()

    # Track time
    start_time = time.time()

    # Run A* search
    path, cost, node_expansion_count = a_star_search_with_multiple_heuristics(graph, start, goal, sld_bucharest, heuristic_type)

    # Measure end time and memory usage
    end_time = time.time()
    execution_time = end_time - start_time
    current_memory, peak_memory = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Print results
    print(f"A* Search with {heuristic_type.upper()} Heuristic:")
    print(f"Path: {path}")
    print(f"Cost: {cost}")
    print(f"Execution Time: {execution_time:.6f} seconds")
    print(f"Memory Usage: {peak_memory / 1024:.2f} KB")
    print(f"Nodes Expanded: {node_expansion_count}")
    print("-------------------------------------------------\n")


# Example Romania map graph
romania_map = {
    'Arad': {'Zerind': 75, 'Timisoara': 118, 'Sibiu': 140},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}




A* Search with SLD Heuristic:
Path: ['Timisoara', 'Arad', 'Sibiu', 'Fagaras', 'Bucharest', 'Urziceni', 'Vaslui']
Cost: 795
Execution Time: 0.000233 seconds
Memory Usage: 4.08 KB
Nodes Expanded: 24
-------------------------------------------------

A* Search with TRIANGLE Heuristic:
Path: ['Timisoara', 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Craiova', 'Pitesti', 'Bucharest', 'Urziceni', 'Vaslui']
Cost: 950
Execution Time: 0.000247 seconds
Memory Usage: 2.92 KB
Nodes Expanded: 29
-------------------------------------------------



In [24]:
#BFS CODE with performance analysis
from collections import deque
import time
import tracemalloc

def bfs_search(graph, start, goal):
    # Initialize frontier (queue) and other variables
    frontier = deque([start])
    explored = set()
    path = {start: None}

    # Track node expansion count
    node_expansion_count = 0

    while frontier:
        node = frontier.popleft()
        node_expansion_count += 1  # Increment node expansion count when a node is dequeued

        if node == goal:
            return construct_path(path, start, goal), node_expansion_count

        explored.add(node)

        for neighbor in graph[node].keys():  # Explore neighbors
            if neighbor not in explored and neighbor not in frontier:
                frontier.append(neighbor)
                path[neighbor] = node

    return None, node_expansion_count  # Return no path if goal is not found


def analyze_bfs_performance(graph, start, goal):
    tracemalloc.start()
    start_time = time.time()

    path = bfs_search(graph, start, goal)

    end_time = time.time()
    execution_time = end_time - start_time
    current_memory, peak_memory = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print("Breadth-First Search:")
    print(f"Path: {path}")
    print(f"Path Cost: {len(path) - 1 if path else 'N/A'}")
    print(f"Execution Time: {execution_time:.6f} seconds")
    print(f"Memory Usage: {peak_memory / 1024:.2f} KB")
    print("-------------------------------------------------\n")




Breadth-First Search:
Path: ['Timisoara', 'Arad', 'Sibiu', 'Fagaras', 'Bucharest', 'Urziceni', 'Vaslui']
Path Cost: 6
Execution Time: 0.000087 seconds
Memory Usage: 2.20 KB
Nodes Expanded: 17
-------------------------------------------------



In [29]:
#TIME AND SPACE COMPLEXITY of both bfs and A*
def analyze_complexity():
    print("Complexity Analysis:")
    print("Breadth-First Search:")
    print("Time Complexity: O(b^d) where b is branching factor and d is depth of the shallowest solution.")
    print("Space Complexity: O(b^d) because it stores all nodes in memory.")
    print("\nA* Search:")
    print("Time Complexity: O(b^d) in the worst case, similar to BFS.")
    print("Space Complexity: O(b^d) as well, due to storing nodes and costs in the priority queue.")
    print("-------------------------------------------------\n")



analyze_complexity()


Complexity Analysis:
Breadth-First Search:
Time Complexity: O(b^d) where b is branching factor and d is depth of the shallowest solution.
Space Complexity: O(b^d) because it stores all nodes in memory.

A* Search:
Time Complexity: O(b^d) in the worst case, similar to BFS.
Space Complexity: O(b^d) as well, due to storing nodes and costs in the priority queue.
-------------------------------------------------

